# Crash Report Jan 18, 2016

Version: 2016.06.00.8


In [1]:
"""
General import statements
"""
import crash_analysis
import pandas as pd
from matplotlib import pyplot as plt 
%matplotlib inline

In [2]:
"""
Parse zipfiles into dataframe
"""
from crash_analysis.parser import extract_zipfiles, xmldocs_to_dataframe
zipfile_location = 'C:\\CrashReports\\'

extract_zipfiles(zipfile_location)
df = xmldocs_to_dataframe(zipfile_location)

df.drop_duplicates(inplace=True)

In [3]:
"""
Save/load dataframe to/from CSV
"""
from crash_analysis import read_csv

new_data = False
# 201606008
cache_location = 'src/data/201606008_jan24_2pm.csv'

if new_data:
    df.to_csv(cache_location, encoding='utf-8')
else:
    df = read_csv(cache_location)

In [4]:
"""
Get metadata about dataframe
"""
col_names = crash_analysis.get_columns(df)
versions = df['AppVersion'].value_counts()
num_crashes = len(df)


print(col_names)
print(versions)
print(num_crashes)

['ACCDT_Field', 'Active_ClientFileName', 'Active_Field', 'Active_Form', 'Active_FormsetID', 'Active_FormsetVersion', 'AppName', 'AppVersion', 'BasWin16.INI', 'Batch_ClientFileName', 'CrashGUID', 'CrashRpt', 'Current_Calcsection', 'CustNum', 'CustomProps', 'DataFileCount', 'ExceptionAddress', 'ExceptionCode', 'ExceptionModule', 'ExceptionModuleBase', 'ExceptionModuleVersion', 'ExceptionType', 'FileList', 'FormsPrinter', 'GUIResourceCount', 'GeoLocation', 'ImageName', 'InnerException', 'InstallType', 'InvParamExpression', 'InvParamFile', 'InvParamFunction', 'InvParamLine', 'Last_Calcsection', 'ManagedException', 'ManagedException.txt', 'MemoryUsageKbytes', 'Message', 'OSIs64Bit', 'OpenHandleCount', 'OperatingSystem', 'ProWin16.INI', 'ProblemDescription', 'Source', 'StackTrace', 'SystemTimeUTC', 'TimeStamp', 'WorkStationName', 'WorkStationType', 'crashdump.dmp', 'crashrpt.xml', 'empty']
201606008    857
Name: AppVersion, dtype: int64
857


In [5]:
"""
Customer Description Analysis
"""
from crash_analysis.analysis import remove_empty, get_column, stem_frequency
customer_desc = remove_empty(get_column(df, 'ProblemDescription'))

stem_frequency(customer_desc)

print

total words: 878
tri        	:   34 	 [['trying'], ['tried'], ['trying'], ['trying'], ['tried'], ['trying']]
program    	:   28 	 [['program'], ['program'], ['program'], ['program'], ['program'], ['program']]
open       	:   26 	 [['opening'], ['open'], ['opens'], ['open'], ['open'], ['open']]
client     	:   23 	 [['client'], ['client'], ['clients'], ['client'], ['client'], ['clients']]
proseri    	:   21 	 [['proseries'], ['proseries'], ['proseries'], ['proseries'], ['proseries'], ['proseries']]
file       	:   21 	 [['file'], ['file'], ['file'], ['file'], ['file'], ['file']]
work       	:   20 	 [['working'], ['working'], ['work'], ['working'], ['working'], ['working']]
crash      	:   19 	 [['crash'], ['crashed'], ['crashes'], ['crashed'], ['crashing'], ['crashed']]
delet      	:   18 	 [['delete'], ['deleted'], ['deleted'], ['deleting'], ['delete'], ['deleting']]
2016       	:   17 	 [['2016'], ['2016'], ['2016'], ['2016'], ['2016'], ['2016']]
return     	:   16 	 [['return'], ['r

In [9]:
"""
Frequency of error messages and stack traces
"""

from crash_analysis import filter_dataframe


def get_value_from_hist(n, field='Message'):
    field_hist = df[field].value_counts()
    assert n < len(field_hist)
    return field_hist.keys()[n], field_hist.values[n]


def print_parings(n, field1='Message', groupings=['AppName', 'InstallType', 'OperatingSystem']):
    top_key, count = get_value_from_hist(n, field1)
    df_filtered = filter_dataframe(df, **{field1: top_key})
    if type(groupings) is type(list()):
        field2_hist = df_filtered.groupby(groupings)['CustNum'].nunique()
    else:
        field2_hist = df_filtered[groupings].value_counts()

    print("Query {0} \t count: {1} \t {2}: {3}".format(n+1, count, field1, top_key))
    print(field2_hist)
    print(sum(field2_hist))
    print '\n'

# pd.set_option('display.max_rows', 5000)
# pd.set_option('display.height', 5000)
# [print_parings(n, groupings='ProblemDescription') for n in range(5)]
# print_parings(0, groupings='CustNum')
# [print_parings(n, field1='ExceptionModuleBase', groupings='ExceptionAddress') for n in range(1,7)]
[print_parings(n, field1='Message') for n in [1]]

# df[df['CustNum'] == '0614456913']

Query 2 	 count: 47 	 Message: Method not found: 'Int32 System.Runtime.InteropServices.Marshal.SizeOf(!!0)'.
AppName           InstallType  OperatingSystem                                          
ProSeries - 2016  Network      Windows (TM) Vista Home Premium Build 6002                    1
                               Windows 7 Enterprise Build 7601                               1
                               Windows 7 Home Premium Build 7601                             1
                               Windows 7 Professional Build 7601                            11
                               Windows 7 Professional Build 7601 Service Pack 1              2
                               Windows 7 Ultimate Build 7600                                 2
                               Windows Vista (TM) Home Premium Build 6002 Service Pack 2     1
Name: CustNum, dtype: int64
19




[None]

In [79]:
""" Customer Descriptions """

from crash_analysis import remove_empty
# print(one_cust.head())
pd.set_option('display.max_colwidth', 250)
print(sum(remove_empty(df['ProblemDescription']).value_counts()))

35


In [80]:
"""
Associate by keyterm
"""
from crash_analysis.analysis import associate_by_keyterms

associate_by_keyterms(df, 'ProblemDescription', 'Message', min_count=2)
print

Message by Keyterm
keyterm: program
Method not found: 'Void Intuit.Application.ProSeries.OptionSetupWizardAPI.DataModels.AdditionalInfo..ctor(System.String, System.String, System.String, System.String, System.String, System.String, System.String, System.String, System.String, System.String, System.String, System.String)'.    2
Method not found: 'Int32 System.Runtime.InteropServices.Marshal.SizeOf(!!0)'.                                                                                                                                                                                                                      1
External component has thrown an exception.                                                                                                                                                                                                                                                        1
Name: Message, dtype: int64

keyterm: open
External component has thrown an exception